# Session 1<br>Introduction to Q# and the Quantum Development Kit
Dr Chris Granade<br>
Senior Research Software Development Engineer<br>
Microsoft

# What is Q#?

In [101]:
operation DemonstrateTeleportation() : Result {
    using ((msg, here, there) = (Qubit(), Qubit(), Qubit())) {
        Ry(1.23, msg);
        
        H(here);
        CNOT(here, there);
        
        CNOT(msg, here);
        H(msg);
        
        let xCorrection = M(here);
        let zCorrection = M(msg);
        
        if (xCorrection == One) { X(there); }
        if (zCorrection == One) { Z(there); }
        
        Ry(-1.23, there);
        
        return M(there);
    }
}

DemonstrateTeleportation

In [109]:
%simulate DemonstrateTeleportation

Zero

In [110]:
%trace DemonstrateTeleportation

Microsoft.Quantum.IQSharp.Jupyter.DisplayableHtmlElement

We can make this more modular by extracting the actual teleportation step.

In [4]:
operation TeleportMessage(msg : Qubit, there : Qubit) : Unit {
    using (here = Qubit()) {        
        H(here);
        CNOT(here, there);
        
        CNOT(msg, here);
        H(msg);
        
        if (M(here) == One) { X(there); }
        if (M(msg) == One)  { Z(there); }
    }
}

TeleportMessage

In [5]:
operation DemonstrateTeleportation() : Result {
    using ((msg, there) = (Qubit(), Qubit())) {
        Ry(1.23, msg);        
        TeleportMessage(msg, there);        
        Ry(-1.23, there);
        
        return M(there);
    }
}

DemonstrateTeleportation

In [6]:
%simulate DemonstrateTeleportation

Zero

In [7]:
%trace DemonstrateTeleportation

Microsoft.Quantum.IQSharp.Jupyter.DisplayableHtmlElement

In [14]:
operation PrepareEntangledPair(left : Qubit, right : Qubit) : Unit is Adj + Ctl {
    H(left);
    CNOT(left, right);
}

operation TeleportMessage(msg : Qubit, there : Qubit) : Unit {
    using (here = Qubit()) {        
        PrepareEntangledPair(here, there);
        Adjoint PrepareEntangledPair(msg, here);
        
        (M(here) == One ? X | I)(there);
        (M(msg) == One  ? Z | I)(there);
    }
}

PrepareEntangledPair, TeleportMessage

In [81]:
%simulate DemonstrateTeleportation

()

In [82]:
open Microsoft.Quantum.Diagnostics;

operation TeleportPreparedState(preparation : (Qubit => Unit is Adj + Ctl)) : Unit {
    using ((msg, there) = (Qubit(), Qubit())) {
        preparation(msg);        
        TeleportMessage(msg, there);        
        Adjoint preparation(there);
        
        AssertMeasurement([PauliZ], [there], Zero, "Unpreparing state did not result in |0⟩.");
    }
}

TeleportPreparedState

In [83]:
operation DemonstrateTeleportation() : Unit {
    TeleportPreparedState(Ry(1.23, _));
}

DemonstrateTeleportation

In [84]:
%simulate DemonstrateTeleportation

()

In [85]:
operation DemonstrateEntanglementSwapping() : Unit {
    using (reference = Qubit()) {
        TeleportPreparedState(PrepareEntangledPair(reference, _));
    }
}

DemonstrateEntanglementSwapping

In [86]:
%simulate DemonstrateEntanglementSwapping

()

# What is Q#?

> Quantum programs are **classical programs** that can send instructions to and get measurement results back from simulators and quantum devices.

From that view, Q# is a classical language designed to make it easier to write, test, and run quantum programs:

- High-level, puts focus on algorithms
- Portable across simulators and hardware
- **TODO**

Let's take a look at the various features of Q#, and how they help us focus back on quantum algorithms.

## Functions and Operations

Q# programs are broken down into two kinds of subroutines:

- **Functions** represent determinstic classical logic <br/>
  e.g.: `Sin`, `Sqrt`
- **Operations** represent subroutines that can take some action, such as sending instructions to a quantum device <br/>
  e.g.: `H`, `M`, `CNOT`

In [87]:
function Squared(x : Double) : Double {
    return x * x;
}

Squared, SampleQrng

In [89]:
%simulate Squared x=3.14

9.8596

In [ ]:
operation SampleQrng() : Result {
    using (q = Qubit()) {
        return Measure([PauliX], [q]);
    }
}

In [91]:
%simulate SampleQrng

One

## Types in Q\#
### Classical Data

When we declare functions and operations, we need to say what the **type** of each input and output is.

- **Numeric data types**: `Int`, `Double`, `Complex`
- **Boolean**: `Bool` (`true` or `false`)
- **Measurement results**: `Result` (`Zero` or `One`)
- **Pauli operators**: `Pauli` (`PauliI`, `PauliX`, `PauliY`, or `PauliZ`)
- **Ranges of integers**: `Range` (e.g.: `0..9`, `10..-1..1`)
- **Diagnostic strings**: `String` (e.g.: `"Hello!"`)

We can declare new _variables_ of each type by using the `let` keyword:

In [ ]:
function AngleForBias(bias : Double) : Double {
    // Declares probability as a Double.
    let probability = 0.5 + bias;
    return 2.0 * ArcCos(Sqrt(1.0 - probability));
}

## Mutability and Immutability

By default, variables in Q# are **immutable**.

In [8]:
function DoesntWork() : Unit {
    let x = 3; // declare x as an Int
    set x = 10; // error!
}

/snippet_.qs(3,9): error QS6303: An immutable identifier cannot be modified.


If we want to modify the value of a variable later, we can use the `mutable` keyword:

In [9]:
function Works() : Unit {
    mutable x = 3; // declare x as an Int, and allows us to modify it later
    set x = 10;
}

Works

In [10]:
%simulate Works

()

The `set` keyword can also be used with in-place operators, such as `+=`:

In [135]:
function WhereEven(array : Int[]) : Int[] {
    mutable evens = new Int[0];
    for (element in array) {
        if (element % 2 == 0) {
            set evens += [element];
        }
    }
    return evens;
}

function EvensExample() : Unit {
    Message($"{WhereEven([0, 11, 13, 16, 101, 142])}");
}

WhereEven, EvensExample

In [136]:
%simulate EvensExample

[0,16,142]


()

In [125]:
%simulate WhereEven array=[0, 1, 2]

Received invalid parameters. Please fix and try again:
 array: Error converting value "[0," to type 'System.Collections.Generic.List`1[System.Int64]'. Path '', line 1, position 5.


While variables in Q# can be declared as mutable, **values are always immutable**.
You can assign a new value to a mutable variable, but that has no effect on anywhere else that value is used.

In [140]:
function ImmutableValueExample() : Unit {
    mutable a = [3];
    let b = a;
    // Changing a has no effect on the value of b.
    set a = [4, 5];
    
    Message($"a = {a}, b = {b}");
}

ImmutableValueExample

In [141]:
%simulate ImmutableValueExample

a = [4,5], b = [3]


()

## Types in Q\#
### Tuples

Given any sequence of types, you can make a new **tuple** of those types using `()`.

In [15]:
operation SampleEntangledPair() : (Result, Result) {
    using ((left, right) = (Qubit(), Qubit())) {
        PrepareEntangledPair(left, right);
        return (M(left), M(right));
    }
}

SampleEntangledPair

In [16]:
%simulate SampleEntangledPair

(One, One)

### Tuple-in Tuple-out
Every Q# function and operation takes exactly one input, and returns exactly one output.

> 💡 **TIP**: _Singleton–tuple equivalence_
>
> Every tuple with exactly one item is identical to that item on its own. E.g.: `3` and `(3)` are the same value.

In [147]:
function TupleExample() : Unit {
    let summands = (2, 3);
    Message($"{Fst(summands)} + {Snd(summands)} = {PlusI(summands)}");
    let three = ((3));
    Message($"2 + {three} = {PlusI(2, three)}");
}

TupleExample

In [148]:
%simulate TupleExample

2 + 3 = 5
2 + 3 = 5


()

Like Python and F#, all functions and operations in Q# return a value. Often, if an operation has no meaningful value to return, it will return the emtpy tuple `()` of type `Unit`.

> 💡 **TIP**
>
> You can think of the empty tuple as a box with nothing in it; unlike `void` in other languages, there still is a value, it just happens to be empty.

We can make tuples of other types as well:

In [155]:
operation RotateByPiOverTwo() : Unit {
    using (q = Qubit()) {
        let input = (PI() / 2.0, q);
        Ry(input);
        DumpMachine();
        Adjoint Ry(input);
    }
}

RotateByPi

In [156]:
%simulate RotateByPiOverTwo

|0⟩	0.7071067811865476 + 0𝑖
|1⟩	0.7071067811865475 + 0𝑖

()

## Types in Q\#
### Arrays

For any type `'T`, the type `'T[]` represents _arrays_ where each element has type `'T`.

In [19]:
function FavoriteNumbers() : Int[] {
    return [1, 42, 101];
}

FavoriteNumbers

In [20]:
function ExampleData() : (Int, Bool[])[] {
    return [
        (3, [true, false, true]),
        (11, [false, false])
    ];
}

ExampleData

In [21]:
%simulate ExampleData

(3, [True,False,True]), (11, [False,False])

Really, _any_ type.

In [ ]:
open Microsoft.Quantum.Arrays;

function Silly() : Unit[] {
    return ConstantArray(4, ());
}

In [ ]:
%simulate Silly

Like all other values in Q#, arrays are **immutable**; you can't change elements of an array after it has been created.
You can, however, use the copy-and-update operator `w/` to do something very similar:

In [23]:
function ElementaryVector(length : Int, index : Int) : Int[] {
    return ConstantArray(length, 0) // [0, 0, ..., 0]
           w/ index <- 1;           // replace the index'th element with 1.
}

ElementaryVector

In [25]:
%simulate ElementaryVector length=5 index=1

0, 1, 0, 0, 0

Just as with other values in Q#, making a copy with `w/` doesn't affect the original.

In [26]:
function ImmutabilityExample() : Unit {
    mutable data = [PauliI, PauliX, PauliY, PauliZ];
    let newData = data w/ 1 <- PauliZ;
    Message($"old: {data}\nnew: {newData}");
}

ImmutabilityExample

In [27]:
%simulate ImmutabilityExample

old: [PauliI,PauliX,PauliY,PauliZ]
new: [PauliI,PauliZ,PauliY,PauliZ]


()

You can combine `w/` with the `set` keyword to safely use arrays as mutable variables, without arrays themselves being mutable.

In [29]:
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Arithmetic;

operation EstimateHistogram(nQubits : Int) : Int[] {
    mutable histogram = ConstantArray(PowI(2, nQubits), 0);
    for (idxSample in 0..99) {
        using (register = Qubit[nQubits]) {
            ApplyToEach(H, register);
            let index = MeasureInteger(LittleEndian(register));
            set histogram w/= index <- histogram[index] + 1;
        }
    }
    return histogram;
}

EstimateHistogram

In [33]:
%simulate EstimateHistogram nQubits=2

28, 26, 25, 21

## Types in Q\#
### Qubits and Registers

The `using` statement can be used to either ask for a single qubit, or an array of qubits.
Fresh qubits always start off in the $|0\rangle$ state.

In [36]:
open Microsoft.Quantum.Diagnostics;

operation AllocateRegister(nQubits : Int) : Unit {
    using (register = Qubit[nQubits]) {
        DumpMachine();
    }
}

AllocateRegister

In [49]:
%simulate AllocateRegister nQubits=2

|00⟩	1 + 0𝑖

()

In [48]:
%simulate AllocateRegister nQubits=6

|000000⟩	1 + 0𝑖

()

## Control Flow in Q\#
### `Adjoint` and `Controlled`

The `Adjoint` and `Controlled` keywords let you automatically undo or control Q# operations.

In [58]:
operation DemonstrateAdjoint(angle : Double) : Unit {
    using ((left, right) = (Qubit(), Qubit())) {
        Ry(angle, left);
        SWAP(left, right);
        Adjoint Ry(angle, right);
        DumpMachine();
    }
}

DemonstrateAdjoint

In [60]:
%simulate DemonstrateAdjoint angle=0.356

|00⟩	1 + 0𝑖

()

By adding `is Adj` to your operation definitions, you can use `Adjoint` with them as well as with built-in operations like `X` and `Ry`.

In [61]:
operation RotateAboutPlus(angle : Double, target : Qubit) : Unit is Adj {
    H(target);
    R1(angle, target);
    H(target);
}

RotateAboutPlus

In [64]:
operation DemonstrateRotationAboutPlus(angle : Double) : Unit {
    using (q = Qubit()) {
        RotateAboutPlus(angle, q);
        DumpMachine();
        Adjoint RotateAboutPlus(angle, q);
        DumpMachine();
    }
}

DemonstrateRotationAboutPlus

In [65]:
%simulate DemonstrateRotationAboutPlus angle=1.234

|0⟩	0.665232554035865 + 0.4719091046873169𝑖
|1⟩	0.3347674459641352 + -0.4719091046873169𝑖

|0⟩	1.0000000000000002 + 2.998097765362545E-17𝑖

()

Similarly, `Controlled` lets you control quantum operations on another register being in the $|11\cdots 1\rangle$ state.

| Gate | Q# Operation | Q# Shorthand |
|---|---|---|
| $X$ | `X(target)` | - |
| CNOT | `Controlled X([control], target)` | `CNOT(control, target)` |
| CZ | `Controlled Z([control], target)` | `CZ(control, target)` |
| Toffoli | `Controlled X([control1, control2], target)` | `CCNOT(control1, control2, target)` |
| Fredkin | `Controlled SWAP([control], (target1, target2))` | - |


The `Controlled` keyword can be used with any operation that has `is Ctl` in its signature.

## Types in Q\#
### Functions and Operations as Values

With `Adjoint` and `Controlled` aside, we can come back to the last two kinds of types you'll commonly encounter in Q#:

functions and operations themselves.

In [52]:
operation ApplyTwiceCA<'T>(op : ('T => Unit is Adj + Ctl), input : 'T) : Unit is Adj + Ctl {
    op(input);
    op(input);
}

ApplyTwiceCA

The type `'T => Unit is Adj + Ctl` represents any operation that returns `Unit`, is adjointable, and controllable.

In [66]:
operation DoNothing(target : Qubit) : Unit is Adj + Ctl {
    ApplyTwiceCA(H, target);
}

DoNothing

### Partial application

Q# provides one more tool to make it easier to use functions and operations together.

If you give _part_ of the input to a function or operation, you get back a new function or operation that takes the remainder of the input.

In [69]:
open Microsoft.Quantum.Convert;

function PiOverNRotation(denominator : Int) : (Qubit => Unit is Adj + Ctl) {
    return Ry(PI() / IntAsDouble(denominator), _);
}

PiOverNRotation

In [72]:
operation DemonstratePiOverNRotation(denominator : Int) : Unit  {
    let rotateByPiOverN = PiOverNRotation(denominator);
    using (q = Qubit()) {
        rotateByPiOverN(q);
        DumpMachine();
        Adjoint rotateByPiOverN(q);
    }
}

DemonstratePiOverNRotation

In [76]:
%simulate DemonstratePiOverNRotation denominator=7

|0⟩	0.9749279121818236 + 0𝑖
|1⟩	0.2225209339563144 + 0𝑖

()

By passing operations to other functions and operations, and by using partial application, we can build powerful control patterns quickly.

In [77]:
operation AssertHTwiceDoesNothing() : Unit {
    AssertOperationsEqualReferenced(1,
        ApplyToFirstQubitCA(ApplyTwiceCA(H, _), _),
        ApplyToFirstQubitCA(I, _)
    );
}

AssertHTwiceDoesNothing

In [78]:
%simulate AssertHTwiceDoesNothing

()

## Control Flow in Q\#
### `within` and `apply`

Patterns like $U^\dagger V U$ are very common in quantum computing, and are specially supported in Q#.

In [150]:
operation ReflectAboutUniformSuperposition(register : Qubit[]) : Unit is Adj + Ctl {
    within {
        // Map |++⋯+⟩ to |00⋯0⟩.
        ApplyToEachCA(H, register);
        // Map |00⋯0⟩ to |11⋯1⟩.
        ApplyToEachCA(X, register);
    } apply {
        // Reflect about |11⋯1⟩.
        Controlled Z(Most(register), Tail(register));
    }
    // The mappings |00⋯0⟩ → |11⋯1⟩ and |++⋯+⟩ → |00⋯0⟩ are automatically
    // undone after the apply block.
}

ReflectAboutUniformSuperposition

In [151]:
operation DemoReflection() : Unit {
    using (register = Qubit[3]) {
        within {
            ReflectAboutUniformSuperposition(register);
        } apply {
            DumpMachine();
        }
    }
}

DemoReflection

In [152]:
%simulate DemoReflection

|000⟩	0.7500000000000002 + 0𝑖
|001⟩	-0.25000000000000006 + 0𝑖
|010⟩	-0.25000000000000006 + 0𝑖
|011⟩	-0.25000000000000006 + 0𝑖
|100⟩	-0.25000000000000006 + 0𝑖
|101⟩	-0.25000000000000006 + 0𝑖
|110⟩	-0.25000000000000006 + 0𝑖
|111⟩	-0.25000000000000006 + 0𝑖

()

## Control Flow in Q\#
### The Q# Standard Library

The Q# standard library provides a wide range of different functions and operations to help make writing out control flow patterns easier.

In [81]:
operation ApplyParity(controls : Qubit[], target : Qubit) : Unit is Adj + Ctl {
    ApplyToEachCA(CNOT(_, target), controls);
}

ApplyParity

In [85]:
operation DemonstrateParity(nQubits : Int) : Unit {
    using (register = Qubit[nQubits]) {
        ApplyToEach(H, register);
        using (target = Qubit()) {
            within {
                ApplyParity(register, target);
            } apply {
                Z(target);
            }
        }
        DumpMachine();
        ResetAll(register);
    }
}

DemonstrateParity

In [87]:
%simulate DemonstrateParity nQubits=3

|000⟩	0.35355339059327384 + 0𝑖
|001⟩	-0.35355339059327384 + 0𝑖
|010⟩	-0.35355339059327384 + 0𝑖
|011⟩	0.35355339059327384 + 0𝑖
|100⟩	-0.35355339059327384 + 0𝑖
|101⟩	0.35355339059327384 + 0𝑖
|110⟩	0.35355339059327384 + 0𝑖
|111⟩	-0.35355339059327384 + 0𝑖

()

In [116]:
operation ApplyPairwiseParity(controls : Qubit[], targets : Qubit[]) : Unit is Adj + Ctl {
    ApplyToEachCA(CCNOT, Zip3(Most(controls), Rest(controls), targets));
}

ApplyPairwiseParity

In [120]:
operation DemonstratePairwiseParity(nQubits : Int) : Unit {
    using ((controls, targets) = (Qubit[nQubits], Qubit[nQubits - 1])) {
        ApplyToEach(H, controls);
        ApplyPairwiseParity(controls, targets);
        DumpMachine();
        ResetAll(controls + targets);
    }
}

DemonstratePairwiseParity

In [121]:
%simulate DemonstratePairwiseParity nQubits=3

|00000⟩	0.35355339059327384 + 0𝑖
|00100⟩	0.35355339059327384 + 0𝑖
|01000⟩	0.35355339059327384 + 0𝑖
|01101⟩	0.35355339059327384 + 0𝑖
|10000⟩	0.35355339059327384 + 0𝑖
|10100⟩	0.35355339059327384 + 0𝑖
|11010⟩	0.35355339059327384 + 0𝑖
|11111⟩	0.35355339059327384 + 0𝑖

()

## Control Flow in Q\#
### Conditional Statements and Expressions

TODO

## Control Flow in Q\#
### Repeat-until-success

TODO

## Diagnosing Q# Programs

## Putting it Together: Grover's Search

In [165]:
open Microsoft.Quantum.Samples.IEEEQuantumWeek;

In [159]:
operation MarkItemByIndex(
    idxMarkedItem : Int,
    register : Qubit[],                                             
    flag : Qubit)                                                   
: Unit is Adj + Ctl {
    (ControlledOnInt(idxMarkedItem, X))(register, flag);
}

MarkItemByIndex

In [163]:
operation DemoMarkedItem() : Unit {
    using ((register, flag) = (Qubit[3], Qubit())) {
        within {
            ApplyToEachCA(H, register);
            MarkItemByIndex(3, register, flag);
        } apply {
            DumpMachine();
        }
    }
}

DemoMarkedItem

In [164]:
%simulate DemoMarkedItem

|0000⟩	0.35355339059327384 + 0𝑖
|0010⟩	0.35355339059327384 + 0𝑖
|0100⟩	0.35355339059327384 + 0𝑖
|0110⟩	0.35355339059327384 + 0𝑖
|1000⟩	0.35355339059327384 + 0𝑖
|1010⟩	0.35355339059327384 + 0𝑖
|1101⟩	0.35355339059327384 + 0𝑖
|1110⟩	0.35355339059327384 + 0𝑖

()

In [173]:
operation SearchForMarkedItem(
    nItems : Int,
    markItem : ((Qubit[], Qubit) => Unit is Adj)
)
: Int {
    using (qubits = Qubit[BitSizeI(nItems)]) {
        PrepareInitialState(qubits);

        DumpMachine();

        for (idxIteration in 0..NIterations(BitSizeI(nItems)) - 1) {
            ReflectAboutMarkedState(markItem, qubits);
            ReflectAboutInitialState(ApplyToEachCA(H, _), qubits);        
        }

        DumpMachine();

        return MeasureInteger(LittleEndian(qubits));
    }
}

SearchForMarkedItem

In [174]:
operation RunGroverSearch(nItems : Int, idxMarkedItem : Int) : Unit {
    let markItem = MarkItemByIndex(idxMarkedItem, _, _);
    let foundItem = SearchForMarkedItem(nItems, markItem);
    Message($"Marked {idxMarkedItem} and found {foundItem}.");
}

RunGroverSearch

In [175]:
%simulate RunGroverSearch nItems=4 idxMarkedItem=1

|000⟩	0.35355339059327384 + 0𝑖
|001⟩	0.35355339059327384 + 0𝑖
|010⟩	0.35355339059327384 + 0𝑖
|011⟩	0.35355339059327384 + 0𝑖
|100⟩	0.35355339059327384 + 0𝑖
|101⟩	0.35355339059327384 + 0𝑖
|110⟩	0.35355339059327384 + 0𝑖
|111⟩	0.35355339059327384 + 0𝑖

|000⟩	-0.08838834764831864 + 0𝑖
|001⟩	-0.08838834764831864 + 0𝑖
|010⟩	-0.08838834764831864 + 0𝑖
|011⟩	-0.08838834764831861 + 0𝑖
|100⟩	0.9722718241315038 + 0𝑖
|101⟩	-0.08838834764831864 + 0𝑖
|110⟩	-0.0883883476483187 + 0𝑖
|111⟩	-0.08838834764831863 + 0𝑖

Marked 1 and found 1.


()